<a href="https://colab.research.google.com/github/royn5618/ComputerVisionProject/blob/master/Car_Damage_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## VGG 16

* Trained on ILSVR (ImageNet)
* VGG 16 - based on CNN architecture
* Model calculates the probablity between 0 to 1 for each label
* Choose the category with the highest probabality

## VGG 16 Architecture

16 layers with learnable weights

{conv1 conv2 pooling} x 2
{conv1 conv2 conv3 pooling} x 3
{dense} x 3

Thus, 13 conv layers - 4 + 9 = 13 


## Transfer Learning

Instead creating a model from scratch, utilize the weights of a pre-trained model.
-saves infrastructural expenses
-saves time
The learning is transfered.


In [ ]:
import numpy as np

import matplotlib.pyplot as plt

from keras.applications import VGG16
from keras.applications.vgg16 import decode_predictions
from keras.preprocessing.image import img_to_array, load_img
from keras.applications.imagenet_utils import preprocess_input
# from keras.applications.vgg16 import preprocess_input

In [ ]:
model = VGG16(weights='imagenet')

In [ ]:
model.summary()

In [ ]:
def prep_image(img_path):
    img = load_img(img_path, target_size=(224, 224))
    x = img_to_array(img)
    x = np.expand_dims(
        x, axis=0
    )  # image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    # print(x.shape)
    x = preprocess_input(x)
    return x

In [ ]:
test_img_path = '/content/drive/My Drive/London/IMG_20190720_084522.jpg'

In [ ]:
prepped_image = prep_image(test_img_path)

In [ ]:
model.predict(prepped_image)

In [ ]:
probabilities = model.predict(prepped_image)

In [ ]:
probabilities.shape

In [ ]:
# Return a tuple (class_name, class_description, score)
decode_predictions(probabilities, top=5)

In [ ]:
img = load_img(test_img_path, target_size=(224, 224))
plt.imshow(img)

In [ ]:
!wget http://imagenet.stanford.edu/internal/car196/cars_train.tgz

In [ ]:
!tar -xvzf cars_train.tgz

In [ ]:
import os
all_car_images=os.listdir('cars_train')

In [ ]:
train_folder_path = 'cars_train'

In [ ]:
all_car_images[:5]

In [ ]:
from collections import Counter, defaultdict


def get_car_labels(img_train_directory):
    counter_car_vgg_labels_dict = defaultdict(float)
    all_car_images = os.listdir(img_train_directory)
    for i, car_image in enumerate(all_car_images):
        img = prep_image(img_train_directory + '/' + car_image)
        label_probabilities = model.predict(img)
        get_top_predition_labels = decode_predictions(label_probabilities,
                                                      top=5)
        for each_top_label in get_top_predition_labels[0]:
            counter_car_vgg_labels_dict[
                each_top_label[0:2]] += each_top_label[2]
    return Counter(counter_car_vgg_labels_dict)

In [ ]:
car_counter = get_car_labels(train_folder_path)

In [ ]:
car_counter

In [ ]:
cat_list  = [k for k, v in car_counter.most_common()[:27]]

In [ ]:
cat_list

In [ ]:
import pickle

with open('counter_car_vgg_labels_dict.pk', 'wb') as pickle_file:
    pickle.dump(car_counter, pickle_file)

In [ ]:
!wget http://imagenet.stanford.edu/internal/car196/cars_test.tgz
!tar -xvzf cars_test.tgz

In [ ]:
# Everything in the cars_test must be classified as "CAR"
# Wil test the original test image if that is a car or not
test_img_path

 '

In [ ]:
def eval_car_or_not(image_path, model, top_cat_list):
    img = prep_image(image_path)
    out = model.predict(img)
    top = decode_predictions(out, top=5)
    print("Validating that this is a picture of your car...")
    for j in top[0]:
        if j[0:2] in top_cat_list:
            print (j[0:2])
            print("Validation complete - proceed to damage evaluation")
            return 1
    print("Are you sure this is a picture of your car? Please take another picture (try a different angle or lighting) and try again.")
    return 0

In [ ]:
eval_car_or_not(test_img_path, model, cat_list)

In [ ]:
test_car_image = 'cars_test/07699.jpg'

In [ ]:
eval_car_or_not(test_car_image, model, cat_list)

In [ ]:
test_images = os.listdir('cars_test')[:10]

In [ ]:
for each_car_image in test_images:
    eval_car_or_not('cars_test' + '/' + each_car_image, model, cat_list)

In [ ]:
test_images_not_car = os.listdir('/content/drive/My Drive/London All')[:10]

In [ ]:
for each_not_car_image in test_images_not_car:
    test_non_car_img_path = '/content/drive/My Drive/London All' + '/' + each_not_car_image
    imgtest = load_img(test_non_car_img_path, target_size=(224, 224))
    plt.imshow(imgtest)
    plt.show()
    eval_car_or_not(test_non_car_img_path, model, cat_list)